In [1]:
from z3 import *

In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.tree import _tree, plot_tree
import pandas as pd
from matplotlib import pyplot as plt
import pickle

In [3]:
from utils import *

In [4]:
data = load_breast_cancer()
feature_names = data["feature_names"]

In [5]:
X = data['data']
y = data["target"] == 1
feature_names = data["feature_names"]
ntrees = 50
clf = RandomForestClassifier(n_estimators=ntrees, max_depth=3).fit(X, y)

In [6]:
clf.score(X, y)

0.9736379613356766

In [7]:
all_thresh = get_ens_thresh(clf)
data_ = disc_ens_data(X, all_thresh)
print(data_.shape)
data_.head()

(569, 289)


,12_1.2944999933242798,12_1.3974999785423279,13_30.605000495910645,23_952.8999938964844,22_117.44999694824219,4_0.10850000008940697,4_0.10734999924898148,4_0.08194000273942947,9_0.052810000255703926,4_0.07471000030636787,...,7_0.051740000024437904,28_0.26010000705718994,7_0.051455000415444374,7_0.050325000658631325,7_0.054099999368190765,7_0.03419000096619129,7_0.05176999978721142,7_0.07919000089168549,7_0.05960499867796898,10_0.2216000035405159
0,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,False,False,True,True,True,...,True,True,True,True,True,True,True,False,True,True
2,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
3,True,True,False,False,False,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,False,False,True,True,True,...,True,False,True,True,True,True,True,True,True,True


In [8]:
var_x = create_var_x(all_thresh)

# Soft Solution

In [9]:
epsilon = 0.4

index = 105
sample = X[index:index+1, :]
clf.predict_proba(sample)

array([[0.72444963, 0.27555037]])

In [10]:
s, c_weights = soft_attack(clf, sample, epsilon, var_x)

False
unknown


In [11]:
adv_weights = get_output(s, c_weights)
np.mean([v for k,v in adv_weights.items()])

0.823100544603881

In [12]:
clf.predict_proba(sample)[0][1]

0.27555037442608055

In [13]:
x_adv, x_adv_sample, compare = get_x_adv(s, var_x, sample)
compare.sort_values(by=["diff(%)"], ascending=False)

,0,1,diff(%),bound
7,0.096010,0.057959,0.396327,"[0.056619999930262566, 0.05795999988913536]"
13,34.660000,21.087245,0.391597,"[18.335000038146973, 21.09000015258789]"
16,0.054730,0.033711,0.384055,"[0.029385000467300415, 0.03371500037610531]"
10,0.390800,0.242979,0.378253,"[0.2216000035405159, 0.24300000071525574]"
14,0.007162,0.009780,0.365570,"[0.009775999933481216, 0.013989999890327454]"
29,0.140500,0.093638,0.333538,"[0.07158999890089035, 0.09365999698638916]"
27,0.198600,0.134897,0.320758,"[0.13234999775886536, 0.13490000367164612]"
3,530.200000,699.050618,0.318466,"[699.0500183105469, 699.6499938964844]"
25,0.409900,0.281523,0.313191,"[0.2543499991297722, 0.28155000507831573]"
21,22.400000,15.419565,0.311627,"[-inf, 15.434999942779541]"


In [14]:
clf.predict_proba(x_adv_sample)[0][1]

0.8231005446038807

# Hard Solution

In [15]:
epsilon = 0.3

nbits = 8

index = 105
sample = X[index:index+1, :]

In [16]:
s, c_weights, seq_num = hard_attack(clf, sample, epsilon, var_x, nbits)

False
sat


In [17]:
clf.predict_proba(sample)[0][1]

0.27555037442608055

In [18]:
get_value(s.model(), seq_num, nbits , ntrees)

0.5011764705882353

In [19]:
adv_weights = get_output(s, c_weights)
np.mean([v for k,v in adv_weights.items()])

0.5023688462764473

In [20]:
x_adv, x_adv_sample, compare = get_x_adv(s, var_x, sample)
compare.sort_values(by=["diff(%)"], ascending=False)

,0,1,diff(%),bound
27,0.198600,0.141696,0.286525,"[0.13774999976158142, 0.14169999957084656]"
4,0.139800,0.100447,0.281498,"[0.09707499668002129, 0.10044999793171883]"
25,0.409900,0.307574,0.249637,"[0.28155000507831573, 0.3076000064611435]"
7,0.096010,0.074081,0.228408,"[0.05960499867796898, 0.07409500330686569]"
24,0.186200,0.144094,0.226135,"[0.13780000060796738, 0.14410000294446945]"
8,0.192500,0.155347,0.193002,"[0.15244999527931213, 0.155349999666214]"
9,0.076920,0.062870,0.182658,"[0.052810000255703926, 0.0628800019621849]"
12,2.410000,2.033863,0.156073,"[1.3974999785423279, 2.034499943256378]"
17,0.013880,0.011721,0.155575,"[0.0073379999957978725, 0.011725000105798244]"
13,34.660000,29.759235,0.141395,"[24.0, 29.765000343322754]"


In [21]:
clf.predict_proba(x_adv_sample)[0][1]

0.5023688462764472